# B⁺ Mass Fitting Analysis - Blocks Data

This notebook performs mass fitting analysis on B⁺ meson decay data using ROOT and RooFit. 

## Analysis Overview

• **B⁺ → D⁰π⁺ Fitting**: Single-mode mass fits using double Crystal Ball PDFs with exponential background
• **B⁺ → J/ψK⁺ + J/ψπ⁺ Combined Fitting**: Simultaneous two-signal fits separating correctly identified and misidentified decays
• **Block-level Analysis**: Processes multiple ROOT files containing different data-taking periods (blocks)
• **Results Storage**: Fit parameters and yields stored in TTree branches for further analysis
• **Visualization**: Automated generation of linear and log-scale mass plots with component separation

## Data Structure

Input files are organized in `data/processed_clean_bp_p/` directory and contain `ST-b2oc` (D⁰π⁺) or `ST-b2cc` (J/ψK⁺/π⁺) trees with reconstructed B⁺ mass information.

In [13]:
import ROOT as r
import numpy as np
import matplotlib.pyplot as plt
from uncertainties import ufloat
from pathlib import Path
from array import array
import re
import os

# Define data directories for organized file management
DATA_CLEAN = Path("data/processed_clean_bp_p")  # Directory containing processed ROOT files

plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
print(r.gROOT.GetVersion())

%jsroot on

c = r.TCanvas()

6.34.04


# B⁺ → D⁰π⁺ Mass Fitting (Blocks)

Perform mass fits to B⁺ → D⁰π⁺ signals in the B⁺ mass spectrum using double Crystal Ball PDFs with exponential background.

## Important notes

• **Input**: ROOT files containing `ST-b2oc` trees with `Bp_DTF_OwnPV_MASS` branch (B2OC mode)
• **Output**: Fit results stored in `fit_results` TTree within each input file  
• **Models**: 
  - **B⁺ signal**: Double Crystal Ball around ~5278 MeV (B⁺ → D⁰π⁺ peak)
  - **Background**: Exponential decay function
• **Branches created**: Shape parameters, yields, uncertainties, and fit quality metrics for B⁺ signal
• **Plotting**: Linear and log-scale mass plots with component separation saved as TCanvas objects

In [14]:
r.EnableImplicitMT() # enable multi-threading in RooFit

# Input files to process
files = [str(DATA_CLEAN/"2024_B2OC_B5.root"), str(DATA_CLEAN/"2024_B2OC_B6.root"), str(DATA_CLEAN/"2024_B2OC_B7.root"), str(DATA_CLEAN/"2024_B2OC_B8.root")]

# Observable: B+ mass variable MeV/c² range
x     = r.RooRealVar("Bp_DTF_OwnPV_MASS", "B^{+} mass", 5200, 5450)    

# B+ → D⁰π+ signal parameters (main peak around 5278 MeV)
mean = r.RooRealVar("mean", "mean", 5278.46, 5250, 5300)

# Crystal Ball 1 parameters
alpha = r.RooRealVar("alpha", "alpha", 1.25015, 1, 2)
n = r.RooRealVar("n", "n", 2.40748, 0.5, 5)
cb_sigma = r.RooRealVar("cb_sigma", "cb_sigma", 11, 1.0, 30)  
crystal_ball = r.RooCBShape("crystal_ball", "Crystal ball PDF", x, mean, cb_sigma, alpha, n)

# Background model
tau = r.RooRealVar("tau", "Decay constant", -0.00219122, -1, 0) 
background = r.RooExponential("background", "Exponential background", x, tau)

# Crystal Ball 2 parameters
alpha_2 = r.RooRealVar("alpha2", "alpha2", -2.2701, -20, -0.01)
n_2 = r.RooRealVar("n2", "n2", 2.47972, 0.05, 50)
cb_sigma_2 = r.RooRealVar("cb_sigma2", "cb_sigma2", 12.7184, 1.0, 20)   
crystal_ball_2 = r.RooCBShape("crystal_ball2", "Crystal ball PDF 2", x, mean, cb_sigma_2, alpha_2, n_2)

frac_cb_2 = r.RooRealVar("frac_cb_2", "Fraction of crystal ball 2", 0.65, 0.0, 1.0)

# Parameter constraints: fix shape parameters, float yields and resolutions
mean.setConstant(False)          # allow mean to float
alpha.setConstant(True)          # fix tail parameter
n.setConstant(True)              # fix tail parameter
cb_sigma.setConstant(True)       # fix resolution
tau.setConstant(False)           # allow background slope to float
alpha_2.setConstant(True)        # fix tail parameter
n_2.setConstant(True)            # fix tail parameter
cb_sigma_2.setConstant(False)    # allow resolution to float
frac_cb_2.setConstant(False)     # allow CB fraction to float

# Helper functions for branch management
def make_branch(tree, name, var_type='d'):
    """Helper function to create a branch with array buffer"""
    if var_type == 'd':
        buf = array('d', [0.])
        tree.Branch(name, buf, f"{name}/D")
    elif var_type == 'i':
        buf = array('i', [0])
        tree.Branch(name, buf, f"{name}/I")
    return buf

def setaddr(tree, name, buf): 
    """Helper function to set branch addresses safely"""
    branch = tree.GetBranch(name)
    if branch: 
        branch.SetAddress(buf)

# Main fitting loop over input files
for fname in files:

    print(f"\n▶ fitting block file {fname}")

    # Open file and get tree, UPDATE keeps existing content
    f = r.TFile.Open(fname, "UPDATE")          
    tree = f.Get("ST-b2oc")
    if not tree:
        print("ST-b2oc tree not found, skipping");  f.Close();  continue

    # Load data into RooDataSet
    data  = r.RooDataSet("data", "data", r.RooArgSet(x), r.RooFit.Import(tree))
    Nevt  = data.numEntries()
    print(f"entries in tree = {Nevt}")

    # Define yield variables with initial estimates
    nsig = r.RooRealVar("nsig", "yield sig", Nevt * 0.25, Nevt * 0.05, Nevt * 0.4)   # ~25% signal
    nbkg = r.RooRealVar("nbkg", "yield bkg", Nevt * 0.75, Nevt * 0.6, Nevt * 0.95)   # ~75% background
    ntot = r.RooFormulaVar("ntot","@0+@1", r.RooArgList(nsig, nbkg))
    
    # Construct composite PDF
    sig_pdf = r.RooAddPdf("sig","", r.RooArgList(crystal_ball_2, crystal_ball,),
                                    r.RooArgList(frac_cb_2))

    # Total model: signal + background
    model = r.RooAddPdf("model","sig+bkg", r.RooArgList(sig_pdf, background),
                                         r.RooArgList(nsig,    nbkg))

    # Perform fit
    fit_res = model.fitTo(data, r.RooFit.Save(True), r.RooFit.Strategy(2))

    # Print fit results
    print("\n=== fitted yields ===")
    print(f"nsig  = {nsig.getVal():.0f} ± {nsig.getError():.0f}")
    print(f"nbkg  = {nbkg.getVal():.0f} ± {nbkg.getError():.0f}")
    print(f"ntot  = {ntot.getVal():.0f} (derived)")
    print(frac_cb_2.getVal())
    
    # Error propagation with uncertainties package
    sig_u = ufloat(nsig.getVal(), nsig.getError()) 
    bkg_u = ufloat(nbkg.getVal(), nbkg.getError())
    frac_bkg = bkg_u / (sig_u + bkg_u)

    print(f"background fraction = {frac_bkg:.6f}")
    print(f"fit status = {fit_res.status()}, covQual = {fit_res.covQual()}")

    # Store results in TTree branches
    f.cd()
    res_tree = f.Get("fit_results")
    
    # Create results tree if it doesn't exist
    if not res_tree:
        res_tree = r.TTree("fit_results", "Mass-fit results per fill")
        
        # Create branches using helper function
        br_mean           = make_branch(res_tree, "mean")
        br_mean_e         = make_branch(res_tree, "mean_err")
        br_alpha          = make_branch(res_tree, "alpha1")
        br_alpha_e        = make_branch(res_tree, "alpha1_err")
        br_n              = make_branch(res_tree, "n1")
        br_n_e            = make_branch(res_tree, "n1_err")
        br_cb_sigma       = make_branch(res_tree, "cb_sigma1")
        br_cb_sigma_e     = make_branch(res_tree, "cb_sigma1_err")
        br_tau            = make_branch(res_tree, "tau")
        br_tau_e          = make_branch(res_tree, "tau_err")
        br_alpha_2        = make_branch(res_tree, "alpha2")
        br_alpha_2_e      = make_branch(res_tree, "alpha2_err")
        br_n_2            = make_branch(res_tree, "n2")
        br_n_2_e          = make_branch(res_tree, "n2_err")
        br_cb_sigma_2     = make_branch(res_tree, "cb_sigma2")
        br_cb_sigma_2_e   = make_branch(res_tree, "cb_sigma2_err")
        br_frac_cb_2      = make_branch(res_tree, "frac_cb_2")
        br_frac_cb_2_e    = make_branch(res_tree, "frac_cb_2_err")
        br_Nevt           = make_branch(res_tree, "Nevt")          
        br_nsig           = make_branch(res_tree, "nsig")
        br_nsig_e         = make_branch(res_tree, "nsig_err")
        br_nbkg           = make_branch(res_tree, "nbkg")
        br_nbkg_e         = make_branch(res_tree, "nbkg_err")
        br_ntot           = make_branch(res_tree, "ntot")
        br_ntot_e         = make_branch(res_tree, "ntot_err")
        
        # Integer branches for fit quality
        br_status         = make_branch(res_tree, "status", 'i')
        br_covqual        = make_branch(res_tree, "covqual", 'i')
        
    else:
        # Tree exists: create arrays and set branch addresses using helper functions
        br_mean           = array('d', [0.]); br_mean_e         = array('d', [0.])
        br_alpha          = array('d', [0.]); br_alpha_e        = array('d', [0.])
        br_n              = array('d', [0.]); br_n_e            = array('d', [0.])
        br_cb_sigma       = array('d', [0.]); br_cb_sigma_e     = array('d', [0.])
        br_tau            = array('d', [0.]); br_tau_e          = array('d', [0.])
        br_alpha_2        = array('d', [0.]); br_alpha_2_e      = array('d', [0.])
        br_n_2            = array('d', [0.]); br_n_2_e          = array('d', [0.])
        br_cb_sigma_2     = array('d', [0.]); br_cb_sigma_2_e   = array('d', [0.])
        br_frac_cb_2      = array('d', [0.]); br_frac_cb_2_e    = array('d', [0.])
        br_Nevt           = array('d', [0.])
        br_nsig           = array('d', [0.]); br_nsig_e         = array('d', [0.])
        br_nbkg           = array('d', [0.]); br_nbkg_e         = array('d', [0.])
        br_ntot           = array('d', [0.]); br_ntot_e         = array('d', [0.])
        br_status         = array('i', [0]); br_covqual        = array('i', [0])

        # Set all branch addresses using dictionary approach with helper function
        branch_mapping = {
            "mean": br_mean, "mean_err": br_mean_e,
            "alpha1": br_alpha, "alpha1_err": br_alpha_e,
            "n1": br_n, "n1_err": br_n_e,
            "cb_sigma1": br_cb_sigma, "cb_sigma1_err": br_cb_sigma_e,
            "tau": br_tau, "tau_err": br_tau_e,
            "alpha2": br_alpha_2, "alpha2_err": br_alpha_2_e,
            "n2": br_n_2, "n2_err": br_n_2_e,
            "cb_sigma2": br_cb_sigma_2, "cb_sigma2_err": br_cb_sigma_2_e,
            "frac_cb_2": br_frac_cb_2, "frac_cb_2_err": br_frac_cb_2_e,
            "Nevt": br_Nevt,
            "nsig": br_nsig, "nsig_err": br_nsig_e,
            "nbkg": br_nbkg, "nbkg_err": br_nbkg_e,
            "ntot": br_ntot, "ntot_err": br_ntot_e,
            "status": br_status, "covqual": br_covqual
        }
        
        for branch_name, buffer in branch_mapping.items():
            setaddr(res_tree, branch_name, buffer)

    # Fill branch values with fit results
    br_mean[0]            = mean.getVal()
    br_mean_e[0]          = mean.getError()
    br_alpha[0]           = alpha.getVal()
    br_alpha_e[0]         = alpha.getError()
    br_n[0]               = n.getVal()
    br_n_e[0]             = n.getError()
    br_cb_sigma[0]        = cb_sigma.getVal()
    br_cb_sigma_e[0]      = cb_sigma.getError()
    br_tau[0]             = tau.getVal()
    br_tau_e[0]           = tau.getError()
    br_alpha_2[0]         = alpha_2.getVal()
    br_alpha_2_e[0]       = alpha_2.getError()
    br_n_2[0]             = n_2.getVal()
    br_n_2_e[0]           = n_2.getError()
    br_cb_sigma_2[0]      = cb_sigma_2.getVal()
    br_cb_sigma_2_e[0]    = cb_sigma_2.getError()
    br_frac_cb_2[0]       = frac_cb_2.getVal()
    br_frac_cb_2_e[0]     = frac_cb_2.getError()
    br_Nevt[0]            = Nevt
    br_nsig[0]            = nsig.getVal()
    br_nsig_e[0]          = nsig.getError()
    br_nbkg[0]            = nbkg.getVal()
    br_nbkg_e[0]          = nbkg.getError()
    
    ntot_u                = sig_u + bkg_u         
    br_ntot[0]            = ntot_u.n
    br_ntot_e[0]          = ntot_u.s
    br_status[0]          = fit_res.status()
    br_covqual[0]         = fit_res.covQual()

    # Write results to tree
    res_tree.Fill()
    res_tree.Write("", r.TObject.kOverwrite)

    # Create mass plot with all components
    frame = x.frame(r.RooFit.Title(f"{fname} mass fit"))
    data.plotOn(frame, r.RooFit.MarkerStyle(20), r.RooFit.LineColor(r.kWhite), r.RooFit.DrawOption("PE0"))
    model.plotOn(frame, r.RooFit.Components(background), r.RooFit.FillColor(r.kGreen + 2), r.RooFit.FillStyle(3001), r.RooFit.DrawOption("F"), r.RooFit.LineColor(r.kGreen + 2), r.RooFit.LineStyle(r.kDashed))
    data.plotOn(frame, r.RooFit.MarkerStyle(20), r.RooFit.LineColor(r.kBlack), r.RooFit.DrawOption("PE0"))
    model.plotOn(frame, r.RooFit.LineColor(r.kRed), r.RooFit.Name("total_curve"))
    model.plotOn(frame, r.RooFit.Components(crystal_ball), r.RooFit.LineStyle(2), r.RooFit.LineColor(r.kBlue))
    model.plotOn(frame, r.RooFit.Components(crystal_ball_2), r.RooFit.LineStyle(3), r.RooFit.LineColor(r.kBlue))
    
    # Extract block/fill info from filename for plot titles
    match = re.search(r'_B(\d+)(?:_F(\d+))?\.root$', fname)
    block_str = f"Block {match.group(1)}" if match else ""
    fill_str = f"Fill {match.group(2)}" if match and match.group(2) else ""
    title_latex = "B^{+} \\rightarrow \\bar{D}^{0}\\pi^{+}" 

    canvas_name = f"dpi_mass_fit_block{match.group(1)}" if match else f"mass_fit_{fname}"
    plot_title = f"{title_latex} Mass Fit, {block_str}" if match else f"{fname} mass fit"

    # Create and save linear-scale canvas
    c = r.TCanvas(canvas_name, plot_title, 1000, 750)
    frame.SetTitle(plot_title)
    frame.GetXaxis().SetTitle("M(B^{+})  [MeV/c^{2}]")
    frame.GetXaxis().CenterTitle(True)
    frame.GetYaxis().CenterTitle(True)
    frame.Draw()
    
    # Add legend
    legend = r.TLegend(0.72, 0.60, 0.98, 0.88)  # (x1, y1, x2, y2)
    legend.SetTextSize(0.025)
    legend.SetBorderSize(0)
    legend.SetFillStyle(0)
    legend.AddEntry(frame.findObject("total_curve"), "Total", "l")
    legend.AddEntry(frame.findObject("h_data"), "Data", "lep")
    # For background, add a dummy object for the fill
    dummy_bg = r.TH1F("dummy_bg", "", 1, 0, 1)
    dummy_bg.SetFillColor(r.kGreen+2)
    dummy_bg.SetLineColor(r.kGreen+2)
    dummy_bg.SetFillStyle(3001)
    legend.AddEntry(dummy_bg, "Background", "f")
    legend.Draw()
    c.Write(canvas_name) 

    # Automatic y-axis minimum for log scale 
    x_min_bg = 5400 
    x_max_bg = 5450
    n_bins = 100
    bin_width = (x.getMax() - x.getMin()) / n_bins
    n_bg = data.reduce(f"{x.GetName()} >= {x_min_bg} && {x.GetName()} < {x_max_bg}").numEntries()
    ymin = max(1, n_bg / ((x_max_bg - x_min_bg) / bin_width))

    # Create and save log-y version
    frame.SetMinimum(ymin * 0.7)
    c.SetLogy()
    c.Write(canvas_name + "_log")
    
    # Save log-scale plot to output directory
    stem = os.path.splitext(os.path.basename(fname))[0]
    plot_path = DATA_CLEAN / f"dpi_fit_{stem}.png"
    c.SetCanvasSize(2000, 1500)
    c.SaveAs(str(plot_path))
    print(f"saved plot: {plot_path}")

    f.Close()


▶ fitting block file data/processed_clean_bp_p/2024_B2OC_B5.root
entries in tree = 2994691

=== fitted yields ===
nsig  = 866695 ± 1994
nbkg  = 2128013 ± 2288
ntot  = 2994709 (derived)
0.7676424505922872
background fraction = 0.710591+/-0.000522
fit status = 0, covQual = 3
saved plot: data/processed_clean_bp_p/dpi_fit_2024_B2OC_B5.png

▶ fitting block file data/processed_clean_bp_p/2024_B2OC_B6.root
entries in tree = 2443641

=== fitted yields ===
nsig  = 709312 ± 1842
nbkg  = 1734280 ± 2102
ntot  = 2443593 (derived)
0.8298342156543104
background fraction = 0.709726+/-0.000590
fit status = 0, covQual = 3
saved plot: data/processed_clean_bp_p/dpi_fit_2024_B2OC_B6.png

▶ fitting block file data/processed_clean_bp_p/2024_B2OC_B7.root
entries in tree = 2216553

=== fitted yields ===
nsig  = 544306 ± 1739
nbkg  = 1672285 ± 2037
ntot  = 2216591 (derived)
0.8127462138800865
background fraction = 0.754440+/-0.000633
fit status = 0, covQual = 3
saved plot: data/processed_clean_bp_p/dpi_fit_202

Info in <Minuit2>: MnSeedGenerator Computing seed using NumericalGradient calculator
Info in <Minuit2>: MnSeedGenerator Initial state: FCN =      -25468009.68 Edm =       9369.725225 NCalls =     33
Warning in <Minuit2>: MnPosDef Matrix forced pos-def by adding to diagonal 0.0154491
Info in <Minuit2>: MnSeedGenerator run Hesse - Initial seeding state: 
  Minimum value : -25468009.68
  Edm           : 2144952.93
  Internal parameters:	[     0.2356918764      0.304692654     0.1388456842    -0.1433475689     0.1433475689      1.477141165]	
  Internal gradient  :	[     -33713.37252     -4239.112445     -19662.30747     -2869.719205       8609.08075     -349420.5519]	
  Internal covariance matrix:
[[   0.0034051216    0.018362715 -0.00041592636  -0.0011243159   0.0011562961 -1.4624529e-06]
 [    0.018362715    0.099641106  -0.0022554604  -0.0060524754   0.0062245863 -9.3845584e-06]
 [ -0.00041592636  -0.0022554604  5.3266331e-05  0.00013716303 -0.00014106354  1.6483101e-07]
 [  -0.00112431

In [15]:
# B⁺ → D⁰π⁺ Fit Results Inspection
# Select input file to inspect (change as needed)
file_name = DATA_CLEAN/"2024_B2OC_B6.root"  
f = r.TFile.Open(str(file_name))

# Display file contents overview
print("\n--- ROOT file contents -------------------------")
for key in f.GetListOfKeys():
    print(" ", key.GetName(), ":", key.GetClassName())

# Examine fit results TTree structure and contents
print("\n--- fit_results TTree inspection -------------------------")
res_tree = f.Get("fit_results")
if not res_tree:
    print("fit_results tree NOT found!")
else:
    print("\n--- Available branches in fit_results tree --------------")
    for br in res_tree.GetListOfBranches():
        print(" ", br.GetName())
    print(f"\nTotal entries in the tree: {res_tree.GetEntries()}")

    # Display first entry with key fit parameters
    if res_tree.GetEntries() > 0:
        res_tree.GetEntry(0)
        print("\n--- First entry fit results snapshot ---")
        print(f"  B⁺ mass mean    = {res_tree.mean:.2f} ± {res_tree.mean_err:.2f} MeV/c²")
        print(f"  Signal yield    = {res_tree.nsig:.0f} ± {res_tree.nsig_err:.0f}")
        print(f"  Background yield= {res_tree.nbkg:.0f} ± {res_tree.nbkg_err:.0f}")
        print(f"  Total yield     = {res_tree.ntot:.0f} ± {res_tree.ntot_err:.0f}")
        print(f"  CB2 fraction    = {res_tree.frac_cb_2:.4f}")
        print(f"  Fit status      = {res_tree.status} (0=good)")
        print(f"  Cov quality     = {res_tree.covqual} (3=good)")

# Display saved mass fit plot
print("\n--- Mass fit plot display -------------------------")
canvas_name = "dpi_mass_fit_block6_log"  # Adjust block number as needed
c = f.Get(canvas_name)
if c:
    c.Draw()                        
    print(f"Successfully displayed canvas: {canvas_name}")
else:
    print(f"Canvas '{canvas_name}' not found!")
    print("Available canvases:")
    for key in f.GetListOfKeys():
        if "dpi_mass_fit" in key.GetName():
            print(f"  {key.GetName()}")

f.Close()


--- ROOT file contents -------------------------
  ST-b2oc : TTree
  ST-b2oc : TTree
  fit_results : TTree
  dpi_mass_fit_block6 : TCanvas
  dpi_mass_fit_block6_log : TCanvas

--- fit_results TTree inspection -------------------------

--- Available branches in fit_results tree --------------
  mean
  mean_err
  alpha1
  alpha1_err
  n1
  n1_err
  cb_sigma1
  cb_sigma1_err
  tau
  tau_err
  alpha2
  alpha2_err
  n2
  n2_err
  cb_sigma2
  cb_sigma2_err
  frac_cb_2
  frac_cb_2_err
  Nevt
  nsig
  nsig_err
  nbkg
  nbkg_err
  ntot
  ntot_err
  status
  covqual

Total entries in the tree: 1

--- First entry fit results snapshot ---
  B⁺ mass mean    = 5278.82 ± 0.03 MeV/c²
  Signal yield    = 709312 ± 1842
  Background yield= 1734280 ± 2102
  Total yield     = 2443593 ± 2795
  CB2 fraction    = 0.8298
  Fit status      = 0 (0=good)
  Cov quality     = 3 (3=good)

--- Mass fit plot display -------------------------
Successfully displayed canvas: dpi_mass_fit_block6_log


# Combined J/ψK + J/ψπ Mass Fitting (Blocks)

Perform simultaneous fits to B⁺ → J/ψK⁺ and B⁺ → J/ψπ⁺ signals in the B⁺ mass spectrum using double Crystal Ball PDFs with exponential background.

## Important notes

• **Input**: ROOT files containing `ST-b2cc` trees with `Bp_DTF_OwnPV_MASS` branch
• **Output**: Fit results stored in `fit_results` TTree within each input file  
• **Models**: 
  - **J/ψK signal**: Double Crystal Ball around ~5280 MeV (main B⁺ peak)
  - **J/ψπ signal**: Double Crystal Ball around ~5320 MeV (misidentified peak)  
  - **Background**: Exponential decay function
• **Branches created**: Shape parameters, yields, uncertainties, and fit quality metrics for both signals
• **Plotting**: Linear and log-scale mass plots with component separation saved as TCanvas objects

In [20]:
r.EnableImplicitMT()  # allow ROOT to parallelize

# Input files to process
files = [str(DATA_CLEAN/"2024_B2CC_B5.root"), str(DATA_CLEAN/"2024_B2CC_B6.root"), str(DATA_CLEAN/"2024_B2CC_B7.root"), str(DATA_CLEAN/"2024_B2CC_B8.root")]

# Observable: B+ mass variable MeV/c² range
x = r.RooRealVar("Bp_DTF_OwnPV_MASS", "B^{+} mass", 5200, 5450)   

# J/ψK signal parameters (main B+ peak around 5280 MeV)
mean_jpsik = r.RooRealVar("mean_jpsik", "J/ψK mean", 5279.38, 5250, 5300)

# J/ψK Crystal Ball 1 parameters
alpha_jpsik = r.RooRealVar("alpha_jpsik", "J/ψK alpha", 1.59501, 0.1, 2)
n_jpsik = r.RooRealVar("n_jpsik", "J/ψK n", 2.75673, 0.5, 5)
cb_sigma_jpsik = r.RooRealVar("cb_sigma_jpsik", "J/ψK sigma", 7.2753, 5, 10)
crystal_ball_jpsik = r.RooCBShape("crystal_ball_jpsik", "J/ψK Crystal ball PDF",
                                  x, mean_jpsik, cb_sigma_jpsik, alpha_jpsik, n_jpsik)

# J/ψK Crystal Ball 2 parameters
alpha_2_jpsik = r.RooRealVar("alpha2_jpsik", "J/ψK alpha2", -1.29179, -1.5, -0.1)
n_2_jpsik = r.RooRealVar("n2_jpsik", "J/ψK n2", 5.29, 0.5, 10)
cb_sigma_2_jpsik = r.RooRealVar("cb_sigma2_jpsik", "J/ψK sigma2", 8.21279, 5, 10)
crystal_ball_2_jpsik = r.RooCBShape("crystal_ball2_jpsik", "J/ψK Crystal ball PDF 2",
                                    x, mean_jpsik, cb_sigma_2_jpsik, alpha_2_jpsik, n_2_jpsik)

frac_cb_2_jpsik = r.RooRealVar("frac_cb_2_jpsik", "J/ψK CB2 fraction", 0.5, 0.0, 1.0)

# J/ψπ signal parameters (misidentified peak around 5320 MeV)
mean_jpsipi = r.RooRealVar("mean_jpsipi", "J/ψπ mean", 5320.0, 5310, 5330)

# J/ψπ Crystal Ball 1 parameters
alpha_jpsipi = r.RooRealVar("alpha_jpsipi", "J/ψπ alpha", 1.6, 0.1, 2.5)
n_jpsipi = r.RooRealVar("n_jpsipi", "J/ψπ n", 2.8, 0.5, 5.0)
cb_sigma_jpsipi = r.RooRealVar("cb_sigma_jpsipi", "J/ψπ sigma", 7.5, 5, 15)
crystal_ball_jpsipi = r.RooCBShape("crystal_ball_jpsipi", "J/ψπ Crystal ball PDF",
                                   x, mean_jpsipi, cb_sigma_jpsipi, alpha_jpsipi, n_jpsipi)

# J/ψπ Crystal Ball 2 parameters
alpha_2_jpsipi = r.RooRealVar("alpha2_jpsipi", "J/ψπ alpha2", -1.3, -2.0, -0.1)
n_2_jpsipi = r.RooRealVar("n2_jpsipi", "J/ψπ n2", 5.5, 0.5, 10.0)
cb_sigma_2_jpsipi = r.RooRealVar("cb_sigma2_jpsipi", "J/ψπ sigma2", 8.5, 5, 15)
crystal_ball_2_jpsipi = r.RooCBShape("crystal_ball2_jpsipi", "J/ψπ Crystal ball PDF 2",
                                     x, mean_jpsipi, cb_sigma_2_jpsipi, alpha_2_jpsipi, n_2_jpsipi)

frac_cb_2_jpsipi = r.RooRealVar("frac_cb_2_jpsipi", "J/ψπ CB2 fraction", 0.5, 0.0, 1.0)

# Background model
tau = r.RooRealVar("tau", "Decay constant", -0.00107319, -1, 0) 
background = r.RooExponential("background", "Exponential background", x, tau)

# Parameter constraints: fix shape parameters, float yields and resolutions
# J/ψK constraints
mean_jpsik.setConstant(False)        # allow mean to float
alpha_jpsik.setConstant(True)        # fix tail parameter
n_jpsik.setConstant(True)            # fix tail parameter
cb_sigma_jpsik.setConstant(False)    # allow resolution to float
alpha_2_jpsik.setConstant(True)      # fix tail parameter
n_2_jpsik.setConstant(True)          # fix tail parameter
cb_sigma_2_jpsik.setConstant(False)  # allow resolution to float
frac_cb_2_jpsik.setConstant(False)   # allow CB fraction to float

# J/ψπ constraints
mean_jpsipi.setConstant(False)       # allow mean to float
alpha_jpsipi.setConstant(True)       # fix tail parameter
n_jpsipi.setConstant(True)           # fix tail parameter
cb_sigma_jpsipi.setConstant(False)   # allow resolution to float
alpha_2_jpsipi.setConstant(True)     # fix tail parameter
n_2_jpsipi.setConstant(True)         # fix tail parameter
cb_sigma_2_jpsipi.setConstant(False) # allow resolution to float
frac_cb_2_jpsipi.setConstant(False)  # allow CB fraction to float

tau.setConstant(False)               # allow background slope to float

# Main fitting loop over input files
for fname in files:
    print(f"\n▶ fitting Block file {fname}")

    # Open file and get tree
    f = r.TFile.Open(fname, "UPDATE")
    tree = f.Get("ST-b2cc")
    if not tree:
        print("ST-b2cc tree not found, skipping")
        f.Close()
        continue

    # Load data into RooDataSet
    print("Loading data...")
    data = r.RooDataSet("data_full", "data_full", r.RooArgSet(x), r.RooFit.Import(tree))  
    Nevt_full = data.numEntries()
    print(f"entries in full tree = {Nevt_full}")

    # Define yield variables with initial estimates
    njpsik = r.RooRealVar("njpsik", "J/ψK yield", Nevt_full * 0.15, 0, Nevt_full * 0.4)      # ~15% signal
    njpsipi = r.RooRealVar("njpsipi", "J/ψπ yield", Nevt_full * 0.05, 0, Nevt_full * 0.1)    # ~5% mis-ID
    nbkg = r.RooRealVar("nbkg", "Background yield", Nevt_full * 0.8, Nevt_full * 0.5, Nevt_full * 0.95)  # ~80% background

    # Construct composite PDFs
    jpsik_pdf = r.RooAddPdf("jpsik_pdf", "J/ψK signal",
                            r.RooArgList(crystal_ball_2_jpsik, crystal_ball_jpsik),
                            r.RooArgList(frac_cb_2_jpsik))

    jpsipi_pdf = r.RooAddPdf("jpsipi_pdf", "J/ψπ signal",
                             r.RooArgList(crystal_ball_2_jpsipi, crystal_ball_jpsipi),
                             r.RooArgList(frac_cb_2_jpsipi))

    # Total model: two signals + background
    model = r.RooAddPdf("model", "J/ψK + J/ψπ + bkg",
                        r.RooArgList(jpsik_pdf, jpsipi_pdf, background),
                        r.RooArgList(njpsik, njpsipi, nbkg))

    # Perform fit
    print("Starting fit with the model...")
    fit_res = model.fitTo(data, r.RooFit.Save(True), r.RooFit.Strategy(2), r.RooFit.PrintLevel(-1))

    # Print fit results
    print("\n=== fitted yields (CORRECTED) ===")
    print(f"J/ψK     = {njpsik.getVal():.0f} ± {njpsik.getError():.0f}")
    print(f"J/ψπ     = {njpsipi.getVal():.0f} ± {njpsipi.getError():.0f}")
    print(f"Bkg      = {nbkg.getVal():.0f} ± {nbkg.getError():.0f}")

    # Calculate totals
    nsig_total = njpsik.getVal() + njpsipi.getVal()
    nsig_total_err = (njpsik.getError()**2 + njpsipi.getError()**2) ** 0.5
    ntotal = nsig_total + nbkg.getVal()

    print(f"Total sig = {nsig_total:.0f} ± {nsig_total_err:.0f}")
    print(f"Total     = {ntotal:.0f}")
    print(f"J/ψK CB2 fraction = {frac_cb_2_jpsik.getVal():.4f}")
    print(f"J/ψπ CB2 fraction = {frac_cb_2_jpsipi.getVal():.4f}")

    # Error propagation with uncertainties package
    jpsik_u = ufloat(njpsik.getVal(), njpsik.getError())
    jpsipi_u = ufloat(njpsipi.getVal(), njpsipi.getError())
    bkg_u = ufloat(nbkg.getVal(), nbkg.getError())
    total_u = jpsik_u + jpsipi_u + bkg_u

    # Store results in TTree branches
    f.cd()
    res_tree = f.Get("fit_results")

    # Helper function to create TTree branches
    def make_branch(tree, name):
        buf = array('d', [0.0])
        tree.Branch(name, buf, f"{name}/D")
        return buf

    # Create results tree if it doesn't exist
    if not res_tree:
        res_tree = r.TTree("fit_results", "Mass-fit results per fill")

        # J/ψK parameter branches
        br_mean_jpsik = make_branch(res_tree, "mean_jpsik")
        br_mean_jpsik_e = make_branch(res_tree, "mean_jpsik_err")
        br_alpha_jpsik = make_branch(res_tree, "alpha1_jpsik")
        br_alpha_jpsik_e = make_branch(res_tree, "alpha1_jpsik_err")
        br_n_jpsik = make_branch(res_tree, "n1_jpsik")
        br_n_jpsik_e = make_branch(res_tree, "n1_jpsik_err")
        br_cb_sigma_jpsik = make_branch(res_tree, "cb_sigma1_jpsik")
        br_cb_sigma_jpsik_e = make_branch(res_tree, "cb_sigma1_jpsik_err")
        br_alpha_2_jpsik = make_branch(res_tree, "alpha2_jpsik")
        br_alpha_2_jpsik_e = make_branch(res_tree, "alpha2_jpsik_err")
        br_n_2_jpsik = make_branch(res_tree, "n2_jpsik")
        br_n_2_jpsik_e = make_branch(res_tree, "n2_jpsik_err")
        br_cb_sigma_2_jpsik = make_branch(res_tree, "cb_sigma2_jpsik")
        br_cb_sigma_2_jpsik_e = make_branch(res_tree, "cb_sigma2_jpsik_err")
        br_frac_cb_2_jpsik = make_branch(res_tree, "frac_cb_2_jpsik")
        br_frac_cb_2_jpsik_e = make_branch(res_tree, "frac_cb_2_jpsik_err")

        # J/ψπ parameter branches
        br_mean_jpsipi = make_branch(res_tree, "mean_jpsipi")
        br_mean_jpsipi_e = make_branch(res_tree, "mean_jpsipi_err")
        br_alpha_jpsipi = make_branch(res_tree, "alpha1_jpsipi")
        br_alpha_jpsipi_e = make_branch(res_tree, "alpha1_jpsipi_err")
        br_n_jpsipi = make_branch(res_tree, "n1_jpsipi")
        br_n_jpsipi_e = make_branch(res_tree, "n1_jpsipi_err")
        br_cb_sigma_jpsipi = make_branch(res_tree, "cb_sigma1_jpsipi")
        br_cb_sigma_jpsipi_e = make_branch(res_tree, "cb_sigma1_jpsipi_err")
        br_alpha_2_jpsipi = make_branch(res_tree, "alpha2_jpsipi")
        br_alpha_2_jpsipi_e = make_branch(res_tree, "alpha2_jpsipi_err")
        br_n_2_jpsipi = make_branch(res_tree, "n2_jpsipi")
        br_n_2_jpsipi_e = make_branch(res_tree, "n2_jpsipi_err")
        br_cb_sigma_2_jpsipi = make_branch(res_tree, "cb_sigma2_jpsipi")
        br_cb_sigma_2_jpsipi_e = make_branch(res_tree, "cb_sigma2_jpsipi_err")
        br_frac_cb_2_jpsipi = make_branch(res_tree, "frac_cb_2_jpsipi")
        br_frac_cb_2_jpsipi_e = make_branch(res_tree, "frac_cb_2_jpsipi_err")

        # Background and yield branches
        br_tau = make_branch(res_tree, "tau")
        br_tau_e = make_branch(res_tree, "tau_err")
        br_Nevt_original = make_branch(res_tree, "Nevt_original")
        br_Nevt_used = make_branch(res_tree, "Nevt_used")
        br_scale_factor = make_branch(res_tree, "scale_factor")
        br_nsig_total = make_branch(res_tree, "nsig_total")
        br_nsig_total_e = make_branch(res_tree, "nsig_total_err")
        br_njpsik = make_branch(res_tree, "njpsik")
        br_njpsik_e = make_branch(res_tree, "njpsik_err")
        br_njpsipi = make_branch(res_tree, "njpsipi")
        br_njpsipi_e = make_branch(res_tree, "njpsipi_err")
        br_nbkg = make_branch(res_tree, "nbkg")
        br_nbkg_e = make_branch(res_tree, "nbkg_err")
        br_ntot = make_branch(res_tree, "ntot")
        br_ntot_e = make_branch(res_tree, "ntot_err")

        # Fit quality branches
        br_status = array('i', [0])
        br_covqual = array('i', [0])
        res_tree.Branch("status", br_status, "status/I")
        res_tree.Branch("covqual", br_covqual, "covqual/I")

    else:
        # Tree exists: create arrays and set branch addresses
        br_mean_jpsik           = array('d', [0.]); br_mean_jpsik_e       = array('d', [0.])
        br_alpha_jpsik          = array('d', [0.]); br_alpha_jpsik_e      = array('d', [0.])
        br_n_jpsik              = array('d', [0.]); br_n_jpsik_e          = array('d', [0.])
        br_cb_sigma_jpsik       = array('d', [0.]); br_cb_sigma_jpsik_e   = array('d', [0.])
        br_alpha_2_jpsik        = array('d', [0.]); br_alpha_2_jpsik_e    = array('d', [0.])
        br_n_2_jpsik            = array('d', [0.]); br_n_2_jpsik_e        = array('d', [0.])
        br_cb_sigma_2_jpsik     = array('d', [0.]); br_cb_sigma_2_jpsik_e = array('d', [0.])
        br_frac_cb_2_jpsik      = array('d', [0.]); br_frac_cb_2_jpsik_e  = array('d', [0.])

        br_mean_jpsipi          = array('d', [0.]); br_mean_jpsipi_e      = array('d', [0.])
        br_alpha_jpsipi         = array('d', [0.]); br_alpha_jpsipi_e     = array('d', [0.])
        br_n_jpsipi             = array('d', [0.]); br_n_jpsipi_e         = array('d', [0.])
        br_cb_sigma_jpsipi      = array('d', [0.]); br_cb_sigma_jpsipi_e  = array('d', [0.])
        br_alpha_2_jpsipi       = array('d', [0.]); br_alpha_2_jpsipi_e   = array('d', [0.])
        br_n_2_jpsipi           = array('d', [0.]); br_n_2_jpsipi_e       = array('d', [0.])
        br_cb_sigma_2_jpsipi    = array('d', [0.]); br_cb_sigma_2_jpsipi_e= array('d', [0.])
        br_frac_cb_2_jpsipi     = array('d', [0.]); br_frac_cb_2_jpsipi_e = array('d', [0.])

        br_tau = array('d', [0.]); br_tau_e = array('d', [0.])
        br_Nevt_original = array('d', [0.]); br_Nevt_used = array('d', [0.])
        br_scale_factor = array('d', [0.])
        br_nsig_total = array('d', [0.]); br_nsig_total_e = array('d', [0.])
        br_njpsik = array('d', [0.]); br_njpsik_e = array('d', [0.])
        br_njpsipi = array('d', [0.]); br_njpsipi_e = array('d', [0.])
        br_nbkg = array('d', [0.]); br_nbkg_e = array('d', [0.])
        br_ntot = array('d', [0.]); br_ntot_e = array('d', [0.])
        br_status = array('i', [0]); br_covqual = array('i', [0])

        # Set all branch addresses using the global helper function
        for n, b in {
            "mean_jpsik": br_mean_jpsik, "mean_jpsik_err": br_mean_jpsik_e,
            "alpha1_jpsik": br_alpha_jpsik, "alpha1_jpsik_err": br_alpha_jpsik_e,
            "n1_jpsik": br_n_jpsik, "n1_jpsik_err": br_n_jpsik_e,
            "cb_sigma1_jpsik": br_cb_sigma_jpsik, "cb_sigma1_jpsik_err": br_cb_sigma_jpsik_e,
            "alpha2_jpsik": br_alpha_2_jpsik, "alpha2_jpsik_err": br_alpha_2_jpsik_e,
            "n2_jpsik": br_n_2_jpsik, "n2_jpsik_err": br_n_2_jpsik_e,
            "cb_sigma2_jpsik": br_cb_sigma_2_jpsik, "cb_sigma2_jpsik_err": br_cb_sigma_2_jpsik_e,
            "frac_cb_2_jpsik": br_frac_cb_2_jpsik, "frac_cb_2_jpsik_err": br_frac_cb_2_jpsik_e,

            "mean_jpsipi": br_mean_jpsipi, "mean_jpsipi_err": br_mean_jpsipi_e,
            "alpha1_jpsipi": br_alpha_jpsipi, "alpha1_jpsipi_err": br_alpha_jpsipi_e,
            "n1_jpsipi": br_n_jpsipi, "n1_jpsipi_err": br_n_jpsipi_e,
            "cb_sigma1_jpsipi": br_cb_sigma_jpsipi, "cb_sigma1_jpsipi_err": br_cb_sigma_jpsipi_e,
            "alpha2_jpsipi": br_alpha_2_jpsipi, "alpha2_jpsipi_err": br_alpha_2_jpsipi_e,
            "n2_jpsipi": br_n_2_jpsipi, "n2_jpsipi_err": br_n_2_jpsipi_e,
            "cb_sigma2_jpsipi": br_cb_sigma_2_jpsipi, "cb_sigma2_jpsipi_err": br_cb_sigma_2_jpsipi_e,
            "frac_cb_2_jpsipi": br_frac_cb_2_jpsipi, "frac_cb_2_jpsipi_err": br_frac_cb_2_jpsipi_e,

            "tau": br_tau, "tau_err": br_tau_e,
            "Nevt_original": br_Nevt_original, "Nevt_used": br_Nevt_used, "scale_factor": br_scale_factor,
            "nsig_total": br_nsig_total, "nsig_total_err": br_nsig_total_e,
            "njpsik": br_njpsik, "njpsik_err": br_njpsik_e,
            "njpsipi": br_njpsipi, "njpsipi_err": br_njpsipi_e,
            "nbkg": br_nbkg, "nbkg_err": br_nbkg_e,
            "ntot": br_ntot, "ntot_err": br_ntot_e,
        }.items():
            setaddr(res_tree, n, b)
        setaddr(res_tree, "status", br_status)
        setaddr(res_tree, "covqual", br_covqual)

    # Fill branch values with fit results
    # J/ψK parameters
    br_mean_jpsik[0]        = mean_jpsik.getVal()
    br_mean_jpsik_e[0]      = mean_jpsik.getError()
    br_alpha_jpsik[0]       = alpha_jpsik.getVal()
    br_alpha_jpsik_e[0]     = alpha_jpsik.getError()
    br_n_jpsik[0]           = n_jpsik.getVal()
    br_n_jpsik_e[0]         = n_jpsik.getError()
    br_cb_sigma_jpsik[0]    = cb_sigma_jpsik.getVal()
    br_cb_sigma_jpsik_e[0]  = cb_sigma_jpsik.getError()
    br_alpha_2_jpsik[0]     = alpha_2_jpsik.getVal()
    br_alpha_2_jpsik_e[0]   = alpha_2_jpsik.getError()
    br_n_2_jpsik[0]         = n_2_jpsik.getVal()
    br_n_2_jpsik_e[0]       = n_2_jpsik.getError()
    br_cb_sigma_2_jpsik[0]  = cb_sigma_2_jpsik.getVal()
    br_cb_sigma_2_jpsik_e[0]= cb_sigma_2_jpsik.getError()
    br_frac_cb_2_jpsik[0]   = frac_cb_2_jpsik.getVal()
    br_frac_cb_2_jpsik_e[0] = frac_cb_2_jpsik.getError()

    # J/ψπ parameters
    br_mean_jpsipi[0]       = mean_jpsipi.getVal()
    br_mean_jpsipi_e[0]     = mean_jpsipi.getError()
    br_alpha_jpsipi[0]      = alpha_jpsipi.getVal()
    br_alpha_jpsipi_e[0]    = alpha_jpsipi.getError()
    br_n_jpsipi[0]          = n_jpsipi.getVal()
    br_n_jpsipi_e[0]        = n_jpsipi.getError()
    br_cb_sigma_jpsipi[0]   = cb_sigma_jpsipi.getVal()
    br_cb_sigma_jpsipi_e[0] = cb_sigma_jpsipi.getError()
    br_alpha_2_jpsipi[0]    = alpha_2_jpsipi.getVal()
    br_alpha_2_jpsipi_e[0]  = alpha_2_jpsipi.getError()
    br_n_2_jpsipi[0]        = n_2_jpsipi.getVal()
    br_n_2_jpsipi_e[0]      = n_2_jpsipi.getError()
    br_cb_sigma_2_jpsipi[0] = cb_sigma_2_jpsipi.getVal()
    br_cb_sigma_2_jpsipi_e[0]= cb_sigma_2_jpsipi.getError()
    br_frac_cb_2_jpsipi[0]  = frac_cb_2_jpsipi.getVal()
    br_frac_cb_2_jpsipi_e[0]= frac_cb_2_jpsipi.getError()

    # Background and summary results
    br_tau[0]           = tau.getVal()
    br_tau_e[0]         = tau.getError()
    br_Nevt_original[0] = Nevt_full
    br_Nevt_used[0]     = Nevt_full
    br_scale_factor[0]  = 1.0
    br_nsig_total[0]    = nsig_total
    br_nsig_total_e[0]  = nsig_total_err
    br_njpsik[0]        = njpsik.getVal()
    br_njpsik_e[0]      = njpsik.getError()
    br_njpsipi[0]       = njpsipi.getVal()
    br_njpsipi_e[0]     = njpsipi.getError()
    br_nbkg[0]          = nbkg.getVal()
    br_nbkg_e[0]        = nbkg.getError()
    br_ntot[0]          = total_u.n
    br_ntot_e[0]        = total_u.s
    br_status[0]        = fit_res.status()
    br_covqual[0]       = fit_res.covQual()

    # Write results to tree
    res_tree.Fill()
    res_tree.Write("", r.TObject.kOverwrite)

    # Create mass plot with all components
    frame = x.frame(r.RooFit.Title(f"{fname} Combined J/PsiK + J/PsiPi mass fit"))
    data.plotOn(frame, r.RooFit.MarkerStyle(20), r.RooFit.LineColor(r.kBlack),
                r.RooFit.DrawOption("PE0"), r.RooFit.Name("data_hist"))
    model.plotOn(frame, r.RooFit.Components("background"),
                 r.RooFit.FillColor(r.kGreen + 2), r.RooFit.FillStyle(3001),
                 r.RooFit.DrawOption("F"), r.RooFit.LineColor(r.kGreen + 2),
                 r.RooFit.LineStyle(r.kDashed))
    data.plotOn(frame, r.RooFit.MarkerStyle(20), r.RooFit.LineColor(r.kBlack),
                r.RooFit.DrawOption("PE0"))
    model.plotOn(frame, r.RooFit.LineColor(r.kRed), r.RooFit.Name("total_curve"))
    model.plotOn(frame, r.RooFit.Components("jpsik_pdf"),
                 r.RooFit.LineStyle(2), r.RooFit.LineColor(r.kBlue),
                 r.RooFit.Name("jpsik_curve"))
    model.plotOn(frame, r.RooFit.Components("jpsipi_pdf"),
                 r.RooFit.LineStyle(3), r.RooFit.LineColor(r.kMagenta),
                 r.RooFit.Name("jpsipi_curve"))

    # Extract block/fill info from filename for plot titles
    match = re.search(r'_B(\d+)(?:_F(\d+))?\.root$', fname)
    block_str = f"Block {match.group(1)}" if match else ""
    fill_str = f"Fill {match.group(2)}" if match and match.group(2) else ""
    title_latex = r"B^{+} \rightarrow J/\psi K^{+} + J/\psi \pi^{+}"

    canvas_name = (f"combined_mass_fit_block{match.group(1)}_fill{match.group(2)}"
                   if match else f"combined_mass_fit_{re.sub(r'[^a-zA-Z0-9]+','_',fname)}")
    plot_title = (f"{title_latex} Combined Mass Fit, {block_str} {fill_str}"
                  if match else f"{fname} combined mass fit")

    # Create and save linear-scale canvas
    c = r.TCanvas(canvas_name, plot_title, 1000, 750)
    frame.SetTitle(plot_title)
    frame.GetXaxis().SetTitle("M(B^{+})  [MeV/c^{2}]")
    frame.GetXaxis().CenterTitle(True)
    frame.GetYaxis().CenterTitle(True)
    frame.Draw()

    # Add legend
    leg = r.TLegend(0.65, 0.55, 0.98, 0.88)
    leg.SetTextSize(0.025); leg.SetBorderSize(0); leg.SetFillStyle(0)
    leg.AddEntry(frame.findObject("total_curve"), "Total", "l")
    leg.AddEntry(frame.findObject("data_hist"), "Data", "lep")
    leg.AddEntry(frame.findObject("jpsik_curve"), "J/PsiK Signal", "l")
    leg.AddEntry(frame.findObject("jpsipi_curve"), "J/PsiPi Signal", "l")
    dummy_bg = r.TH1F("dummy_bg", "", 1, 0, 1)
    dummy_bg.SetFillColor(r.kGreen+2); dummy_bg.SetLineColor(r.kGreen+2); dummy_bg.SetFillStyle(3001)
    leg.AddEntry(dummy_bg, "Background", "f")
    leg.Draw()
    c.Write(canvas_name)

    # Create log-scale version with appropriate y-axis minimum
    x_min_bg, x_max_bg = 5400, 5450  # background-only region
    n_bins = 100
    bin_width = (x.getMax() - x.getMin()) / n_bins
    n_bg = data.reduce(f"{x.GetName()} >= {x_min_bg} && {x.GetName()} < {x_max_bg}").numEntries()
    ymin = max(1.0, n_bg / ((x_max_bg - x_min_bg) / bin_width))

    frame.SetMinimum(ymin * 0.8)
    c.SetLogy()
    c.Write(canvas_name + "_log")
    
    # Save log-scale plot to output directory
    stem = os.path.splitext(os.path.basename(fname))[0]
    plot_path = DATA_CLEAN / f"dpi_fit_{stem}.png"
    c.SetCanvasSize(2000, 1500)
    c.SaveAs(str(plot_path))
    print(f"saved plot: {plot_path}")

    f.Close()


▶ fitting Block file data/processed_clean_bp_p/2024_B2CC_B7.root
Loading data...
entries in full tree = 14573902
Starting fit with the model...

=== fitted yields (CORRECTED) ===
J/ψK     = 762731 ± 352
J/ψπ     = 61 ± 406
Bkg      = 13825214 ± 354
Total sig = 762792 ± 537
Total     = 14588006
J/ψK CB2 fraction = 0.3955
J/ψπ CB2 fraction = 0.9914
saved plot: data/processed_clean_bp_p/dpi_fit_2024_B2CC_B7.png
[#1] INFO:DataHandling -- RooAbsReal::attachToTree(Bp_DTF_OwnPV_MASS) TTree Float_t branch Bp_DTF_OwnPV_MASS will be converted to double precision.
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data_full) Skipping event #0 because Bp_DTF_OwnPV_MASS cannot accommodate the value 5536.06
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data_full) Skipping event #1 because Bp_DTF_OwnPV_MASS cannot accommodate the value 5168.71
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data_full) Skipping event #2 because Bp_DTF_OwnPV_MASS cannot accommodate the value 5518.3

Warning in <TCanvas::Constructor>: Deleting canvas with same name: combined_mass_fit_block7_fillNone
Info in <TCanvas::Print>: png file data/processed_clean_bp_p/dpi_fit_2024_B2CC_B7.png has been created


In [19]:
import ROOT as r

# Results inspection and visualization
file_name =  DATA_CLEAN/"2024_B2CC_B7.root"   # pick any file
f = r.TFile.Open(str(file_name))

# List all objects stored in the ROOT file
print("\n--- top-level keys in the file -------------------------")
for key in f.GetListOfKeys():
    print(" ", key.GetName(), ":", key.GetClassName())

# Check fit results tree structure and content
res_tree = f.Get("fit_results")
if not res_tree:
    print("fit_results tree NOT found!")
else:
    print("\n--- fit_results branches ------------------------------")
    for br in res_tree.GetListOfBranches():
        print(" ", br.GetName())
    print("\nentries in the tree:", res_tree.GetEntries())

    # Print sample values from first entry
    res_tree.GetEntry(0)
    print("\nfirst entry snapshot:")
    print("  mean_jpsipi   =", res_tree.mean_jpsipi)           
    print("  n1_jpsipi   =", res_tree.n1_jpsipi)               
    print("  alpha1_jpsipi   =", res_tree.alpha1_jpsipi)       
    print("  cb_sigma1_jpsipi   =", res_tree.cb_sigma1_jpsipi) 
    print("  frac_cb_2_jpsipi   =", res_tree.frac_cb_2_jpsipi) 
    print("  n2_jpsipi   =", res_tree.n2_jpsipi)               
    print("  alpha2_jpsipi   =", res_tree.alpha2_jpsipi)       
    print("  cb_sigma2_jpsipi   =", res_tree.cb_sigma2_jpsipi) 
    print("  Nevt_original   =", res_tree.Nevt_original)       
    print("  Nevt_used   =", res_tree.Nevt_used)               
    print("  nsig_total   =", res_tree.nsig_total)             # total signal yield
    print("  njpsik   =", res_tree.njpsik)                     # J/ψK yield
    print("  njpsipi   =", res_tree.njpsipi)                   # J/ψπ yield
    print("  ntot   =", res_tree.ntot)                         # total yield
    print("  nbkg   =", res_tree.nbkg)                         # background yield

# Display saved fit plots
c = f.Get("combined_mass_fit_block7_fillNone_log")             # Adjust block number as needed
if c:
    c.Draw()
else:
    print("canvas object not found; check its name in `f.ls()`")

# Alternative: try linear-scale version
# c_lin = f.Get("combined_mass_fit_block7_fillNone")  # linear-scale canvas


--- top-level keys in the file -------------------------
  ST-b2cc : TTree
  ST-b2cc : TTree
  fit_results : TTree
  combined_mass_fit_block7_fillNone : TCanvas
  combined_mass_fit_block7_fillNone_log : TCanvas

--- fit_results branches ------------------------------
  mean_jpsik
  mean_jpsik_err
  alpha1_jpsik
  alpha1_jpsik_err
  n1_jpsik
  n1_jpsik_err
  cb_sigma1_jpsik
  cb_sigma1_jpsik_err
  alpha2_jpsik
  alpha2_jpsik_err
  n2_jpsik
  n2_jpsik_err
  cb_sigma2_jpsik
  cb_sigma2_jpsik_err
  frac_cb_2_jpsik
  frac_cb_2_jpsik_err
  mean_jpsipi
  mean_jpsipi_err
  alpha1_jpsipi
  alpha1_jpsipi_err
  n1_jpsipi
  n1_jpsipi_err
  cb_sigma1_jpsipi
  cb_sigma1_jpsipi_err
  alpha2_jpsipi
  alpha2_jpsipi_err
  n2_jpsipi
  n2_jpsipi_err
  cb_sigma2_jpsipi
  cb_sigma2_jpsipi_err
  frac_cb_2_jpsipi
  frac_cb_2_jpsipi_err
  tau
  tau_err
  Nevt_original
  Nevt_used
  scale_factor
  nsig_total
  nsig_total_err
  njpsik
  njpsik_err
  njpsipi
  njpsipi_err
  nbkg
  nbkg_err
  ntot
  ntot_err
  st